In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.utils.math import cosine_similarity
import pandas as pd
import numpy as np
import os
import pickle

In [4]:
# OpenAI API Keyを環境変数に保存
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key:')
# 作業ディレクトリの設定
wk_dir = '/content/drive/MyDrive/aozorabunko'

OpenAI API Key:··········


In [5]:
# 埋め込みモデルのインスタンスを作成
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

## テキストの前処理

In [6]:
import shutil # zipファイル展開用
import re
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
os.chdir(wk_dir + '/files')
# remove unneccesary lines and characters from .txt files
# ref: https://www.osaka-kyoiku.ac.jp/~kokugo/nonami/awk/rubycut_awk.html
files = [s for s in os.listdir() if re.search('\.txt$', s) is not None]
texts = []
for file in files:
  with open(file, 'r', encoding='SHIFT_JIS') as f:
    text = f.read()
  text = re.sub('\n-{30,}\n(.|\n)*?\n-{30,}\n', '', text)
  text = re.sub('\n\n\n\n底本：(.|\n)*$', '', text)
  text = re.sub('\n［＃[^］]*?］\n', '\n', text) # ［＃…］だけの行を除く
  text = re.sub('［＃[^］]*?］', '', text) # ［＃…］を除く
  text = re.sub('《[^》]+》', '', text) # ルビを除く
  text = re.sub('｜', '', text) # ルビの境界記号を除く
  texts.append(text)



In [8]:
len(texts)

18

In [9]:
# 作品タイトルリスト（桃太郎は重複）
dict ={}
for text in texts:
    title = text.split('\n')[0] # タイトル
    print(title)


おおかみと七ひきのこどもやぎ
ヘンゼルとグレーテル
星の銀貨
赤ずきんちゃん
ジャックと豆の木
かちかち山
桃太郎
桃太郎
金太郎
瘤とり
お伽草紙
猿かに合戦
舌切りすずめ
浦島太郎
瓜子姫子
文福茶がま
花咲かじじい
一寸法師


## テキストをチャンクに分割

In [10]:
# テキストをチャンクに分割し，Documentのリストを作成
docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=25)
for text in texts:
    title_author = text.split('\n\n')[0] # タイトル・著者
    text = re.sub('^(.|\n)*?\n\n', '', text) # タイトル・著者を除いたテキスト
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        doc = Document(page_content=chunk, metadata={'title_author':title_author})
        docs.append(doc)

len(docs)

874

In [11]:
# 文字数の合計
sum([len(doc.page_content) for doc in docs])

155796

In [12]:
# トークン数の合計
import tiktoken
sum([len(tiktoken.encoding_for_model('text-embedding-ada-002').encode(doc.page_content)) for doc in docs])

167156

## 埋め込みベクトルをndarrayで保存（Chromaを使わない）

In [13]:
# DocumentリストをDataFrameに変換
chunk_df = pd.DataFrame({'title_author':[doc.metadata['title_author'] for doc in docs],
                   'chunk':[doc.page_content for doc in docs]})
chunk_df.head()

,title_author,chunk
0,おおかみと七ひきのこどもやぎ\nDER WOLF UND DIE SIEBEN JUNGEN...,一
1,おおかみと七ひきのこどもやぎ\nDER WOLF UND DIE SIEBEN JUNGEN...,むかし、あるところに、おかあさんのやぎがいました。このおかあさんやぎには、かわいいこどもやぎ...
2,おおかみと七ひきのこどもやぎ\nDER WOLF UND DIE SIEBEN JUNGEN...,「おまえたちにいっておくがね、かあさんが森へ行ってくるあいだ、気をつけてよくおるすばんしてね...
3,おおかみと七ひきのこどもやぎ\nDER WOLF UND DIE SIEBEN JUNGEN...,すると、こどもやぎは、声をそろえて、\n「かあさん、だいじょうぶ、あたいたち、よく気をつけて...
4,おおかみと七ひきのこどもやぎ\nDER WOLF UND DIE SIEBEN JUNGEN...,二


In [14]:
chunk_df.shape

(874, 2)

In [15]:
# DataFrameを保存
chunk_df.to_pickle(wk_dir + '/chunk_df.pkl')

In [ ]:
# チャンクをベクトルに埋め込み，結合してアレイに
embeddings_array = np.array([embeddings.embed_query(chunk) for chunk in chunk_df['chunk']])
embeddings_array.shape

In [ ]:
# 結合したアレイを保存
with open(wk_dir + '/embeddings_array.pkl', 'wb') as f:
  pickle.dump(embeddings_array, f)